# Generador de bits aleatorios

Primero, veamos como crear un qubit en superposicion:

In [1]:
from qiskit import QuantumCircuit, execute, Aer
qubits = QuantumCircuit(1,1)

Ahora veamos como lo podemos poner en superposicion: 

In [2]:
qubits.h(0)

Finalmente, veamos como podemos hacer una medicion: 

In [3]:
qubits.measure(0,0)

Ahora para ejecutar las operaciones anteriores debemos de ejecutar las siguientes lineas:

In [13]:
backend = Aer.get_backend('qasm_simulator')
execute(qubits, backend, shots=1, memory=True).result().get_memory()[0]

'0'

Ahora si lo ponemos en un metodo, quedaria de la siguiente forma:

In [14]:
def get_random_bit():
    qubits = QuantumCircuit(1,1)
    qubits.h(0)
    qubits.measure(0,0)
    return int(execute(qubits, backend, shots=1, memory=True).result().get_memory()[0])

In [22]:
get_random_bit()

1

# Protocolo BB84 para intercambio de llaves.

## Repaso:
Recordemos que el protocolo BB84 funciona de la siguiente manera:
- Alicia elige una cadena aleatoria de bits. Después encripta esa cadena aleatoria seleccionando aleatoriamente una compuerta que puede ser la compuerta Hadamard o la identidad.
- Alicia manda los qubits a Bob que son el resultado del paso anterior.
- Bob aplica un compuerta aleatoria a los qubits que recibe de Alice. La compuerta puede ser una compuerta Hadamard o la identidad.
- Alicia manda a Bob que compuerta uso en cada qubit.
- Bob revisa si Alicia y el usaron la misma compuerta en cada qubit. Si lo hicieron le manda un mensaje a Alicia con el indice qubit en el que aplicaron la misma compuerta.
- Ahora Alicia y Bob tienen una cadena aleatoria compartida.

## Implementacion
Primero emepecemos con unos imports de ciertas bibliotecas que necesitamos.

In [4]:
%matplotlib inline
from qiskit.tools.jupyter import *

En la siguiente celda podemos ver como Alicia prepara una cadena aleatoria y la encripta.

In [32]:
#Computadora de Alicia, Bob no sabe que esta pasando aqui.
m0 = ''.join([str(get_random_bit()) for _ in range(256)])
Alice_bases = [get_random_bit() for _ in range(256)]
qubits = list()
for i in range(len(Alice_bases)):
    mycircuit = QuantumCircuit(1,1)
    if(Alice_bases[i] == 0):
        if(m0[i] == "1"):
            mycircuit.x(0)
    else:
        if(m0[i] == "0"):
            mycircuit.h(0)
        else:
            mycircuit.x(0)
            mycircuit.h(0)
    qubits.append(mycircuit)

En la siguiente celda Bob aplica una compuerta de manera aleatoria a los qubits que recibe de Alice. 

In [33]:
#Computadora de Bob, Alicia no puede ver lo que esta pasando aqui.
Bob_bases = [get_random_bit() for _ in range(256)]
measurements = list()
for i in range(len(Bob_bases)):
    qubit = qubits[i]
    if(Bob_bases[i] == 0):
        qubit.measure(0,0)
    else:
        qubit.h(0)
        qubit.measure(0,0)
    result = execute(qubit, backend, shots=1, memory=True).result()
    measurements.append(int(result.get_memory()[0]))

Después, Alicia le manda a Bob las compuertas que utilizo en cada qubit. Si Bob encuentra que aplicaron las mismas, entonces guarda el indice del qubit en el que las aplicaron.

In [34]:
#Computadora de Bob, Alicia no puede ver lo que esta pasando aqui.
I0 = list()
for i in range(len(Alice_bases)):
    if(Alice_bases[i] == Bob_bases[i]):
        I0.append(i)

Finalmente, ambos toman los indices que tienen en comun y se termina el protocolo. En la siguiente celda se hace una verificacion rapida para ver que realmente ambos tienen la misma cadena compartida.

In [35]:
#Celda de prueba para comprobar que son iguales las llaves
keyAlice = ''.join([m0[x] for x in I0])
keyBob = ''.join([str(measurements[x]) for x in I0])
print(keyAlice)
print(keyBob)
print("La llave de Alicia es igual a la de Bob? " + str(keyAlice == keyBob))

1110100010100001011000111010011101001100110100110000100010100100000011111111111101101101101010001011000001010111110010001000110011
1110100010100001011000111010011101001100110100110000100010100100000011111111111101101101101010001011000001010111110010001000110011
La llave de Alicia es igual a la de Bob? True


## One time-pads

Ahora veamos como se pueden aplicar las llaves anteriores a un cifrado de tipo one time-pads. Pero primero definamos dos funciones para convertir bits a strings y strings a bits.

In [36]:
def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return ''.join([str(x) for x in result])

def frombits(bits):
    chars = []
    for b in range(int(len(bits) / 8)):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

Después definamos la funcion de encriptacion que usará Alicia para encriptar su mensaje "hola". Cabe mencionar que como la llave que generamos es mucho más larga que la cadena que queremos encriptar, solo tomaremos los primeros n bits de la llave para que concida la longitud de esta con la longitud de la cadena.

In [37]:
#Computadora de Alicia
AliceMessage = "hola"
AliceBitsMessage = tobits(AliceMessage)
oneTimePadKey = keyAlice[0:len(AliceBitsMessage)]
output = list()
for bit in range(0, len(AliceBitsMessage)):
    output.append(int(AliceBitsMessage[bit]) ^ int(oneTimePadKey[bit]))
print("El mensaje encriptado es: " + frombits(output))

El mensaje encriptado es: ÎÆ


En la siguiente celda, Bob desencriptara el mensaje de Alice usando su llave y obtendrá la cadena original que Alice queria mandar.

In [38]:
#Computadora de Bob
oneTimePadKey = keyBob[0:len(output)]
original = list()
for bit in range(0, len(AliceBitsMessage)):
    original.append(int(output[bit]) ^ int(oneTimePadKey[bit]))
print("El mensaje desencriptado es: " + frombits(original))

El mensaje desencriptado es: hola
